In [1]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.backends.cudnn as cudnn

from google.colab import drive
drive.mount("/content/drive", force_remount=True)
drive_path = "drive/My Drive/NPEX_2020_Quantization" # do not modify this line
relative_path = "/practice"   # set to your relative path 

base_path = drive_path + relative_path

import sys
sys.path.append(base_path)

from train_test import train, test

LR = 0.01
EPOCH = 5

# Data
print('==> Preparing data')
from dataset import cifar10_dataset
trainloader, testloader = cifar10_dataset(base_path + "/data")

# Model
print('==> Building model')
from resnet_quant import ResNet18
model = ResNet18()
model.load_state_dict(torch.load(base_path + "/train_best.pth"))

if torch.cuda.is_available():
    model.cuda()
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True
criterion = torch.nn.CrossEntropyLoss()

print('==> Full-precision model accuracy')
from quant_op import Q_ReLU, Q_Conv2d, Q_Linear
test(model, testloader, criterion)

for name, module in model.named_modules():
    if isinstance(module, Q_ReLU):
        module.n_lv = 8
        module.bound = 1
    
    if isinstance(module, (Q_Conv2d, Q_Linear)):
        module.n_lv = 8
        module.ratio = 0.5

print('==> Quantized model accuracy')
from quant_op import Q_ReLU, Q_Conv2d, Q_Linear
test(model, testloader, criterion)

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCH, last_epoch=start_epoch-1)

for epoch in range(start_epoch, start_epoch + EPOCH):
    scheduler.step()
    train(model, trainloader, criterion, optimizer, epoch)
    acc = test(model, testloader, criterion)

    if acc > best_acc:
        best_acc = acc
        torch.save(model.module.state_dict(), base_path +  "quant_best.pth")

print('==> Fine-tuned model accuracy')
from quant_op import Q_ReLU, Q_Conv2d, Q_Linear
test(model, testloader, criterion)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
==> Preparing data


Extracting drive/My Drive/NPEX_2020_Quantization/practice/data/cifar-10-python.tar.gz to drive/My Drive/NPEX_2020_Quantization/practice/data
Files already downloaded and verified
==> Building model
==> Full-precision model accuracy
Test: [10/100]	Time 0.028 (0.118)	Loss 0.1693 (0.2120)	Prec@1 95.000 (94.300)
Test: [20/100]	Time 0.027 (0.073)	Loss 0.3840 (0.2106)	Prec@1 93.000 (94.650)
Test: [30/100]	Time 0.032 (0.059)	Loss 0.1171 (0.2190)	Prec@1 96.000 (94.667)
Test: [40/100]	Time 0.031 (0.052)	Loss 0.1930 (0.2174)	Prec@1 97.000 (94.750)
Test: [50/100]	Time 0.029 (0.048)	Loss 0.3337 (0.2224)	Prec@1 94.000 (94.660)
Test: [60/100]	Time 0.027 (0.045)	Loss 0.1739 (0.2162)	Prec@1 96.000 (94.817)
Test: [70/100]	Time 0.027 (0.043)	Loss 0.1721 (0.2082)	Prec@1 96.000 (95.014)
Test: [80/100]	Time 0.026 (0.042)	Loss 0.0803 (0.2097)	Prec@1 96.000 (95.013)
Test: [90/100]	Time 0.026 (0.040)	Loss 0.2229 (0.2079)	Prec@1 94.000 (94.989)
Test: [100/100]	Time 0.026 (0.039)	Loss 0.0745 (0.2060)	Prec@1 97.

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 0: [10/196]	Time 0.217 (0.411)	Data 0.007 (0.062)	Loss 0.0656 (0.0522)	Prec@1 98.047 (98.516)
Epoch 0: [20/196]	Time 0.216 (0.315)	Data 0.002 (0.031)	Loss 0.0180 (0.0502)	Prec@1 100.000 (98.496)
Epoch 0: [30/196]	Time 0.219 (0.283)	Data 0.002 (0.021)	Loss 0.0629 (0.0463)	Prec@1 97.656 (98.607)
Epoch 0: [40/196]	Time 0.219 (0.267)	Data 0.002 (0.016)	Loss 0.0423 (0.0451)	Prec@1 98.828 (98.604)
Epoch 0: [50/196]	Time 0.220 (0.258)	Data 0.002 (0.013)	Loss 0.0764 (0.0463)	Prec@1 97.266 (98.508)
Epoch 0: [60/196]	Time 0.221 (0.251)	Data 0.002 (0.012)	Loss 0.0424 (0.0461)	Prec@1 99.219 (98.516)
Epoch 0: [70/196]	Time 0.225 (0.247)	Data 0.001 (0.010)	Loss 0.0601 (0.0481)	Prec@1 98.047 (98.454)
Epoch 0: [80/196]	Time 0.222 (0.244)	Data 0.002 (0.009)	Loss 0.0449 (0.0477)	Prec@1 98.438 (98.481)
Epoch 0: [90/196]	Time 0.220 (0.241)	Data 0.002 (0.008)	Loss 0.0526 (0.0489)	Prec@1 98.438 (98.424)
Epoch 0: [100/196]	Time 0.222 (0.240)	Data 0.001 (0.008)	Loss 0.0406 (0.0488)	Prec@1 99.219 (98.422

93.66